In [1]:
import tkinter as tk
from tkinter import ttk
oka = "coded by Ok@Nikolov"


class PreisSchätzer:
    # Workaround: Die Kostenfaktoren aus meiner habe ich genutzt um Dictonarys zu erstellen, die dann Auswirkungen auf den Preis haben. 
    bundesland_kostenfaktoren = {
        'Baden-Württemberg': 1.5, 'Bayern': 1.7, 'Berlin': 2.1, 'Brandenburg': 1.1, 
        'Bremen': 1.2, 'Hamburg': 2.5, 'Hessen': 1.3, 'Mecklenburg-Vorpommern': 0.9, 
        'Niedersachsen': 1, 'Nordrhein-Westfalen': 1.1, 'Rheinland-Pfalz': 1, 
        'Saarland': 0.7, 'Sachsen': 0.7, 'Sachsen-Anhalt': 0.6, 
        'Schleswig-Holstein': 1.4, 'Thüringen': 0.6
    }
    
    lage_kostenfaktoren = {'Stadt': 2, 'Land': 1}
    
    ausstattungs_kostenfaktoren = {
        'Rohbau': 0.5, 'Sanierungsbedarf': 0.8, 'Renovierungsbedarf': 0.9, 
        'Einfach': 1, 'Gehoben': 2
    }
    
    hausart_kostenfaktoren = {
        'Einfamilienhaus': 1, 'Doppelhaushälfte': 0.8, 'Mehrfamilienhaus': 0.7
    }

    # Berechnungsmethode basierend auf den verschiedenen Kostenfaktoren
    def berechne_preis(self, eingaben):
        # Basispreise
        grundstück_preis = eingaben['grundstück'] * 160  # Preis pro m² Grundstück
        wohnfläche_preis = eingaben['wohnfläche'] * 2500  # Preis pro m² Wohnfläche
        
        # Bundesland-Kostenfaktor
        bundesland_faktor = self.bundesland_kostenfaktoren[eingaben['bundesland']]
        
        # Lage-Kostenfaktor (Stadt oder Land)
        lage_faktor = self.lage_kostenfaktoren[eingaben['lage']]
        
        # Ausstattung-Kostenfaktor
        ausstattungs_faktor = self.ausstattungs_kostenfaktoren[eingaben['ausstattung']]
        
        # Hausart-Kostenfaktor
        hausart_faktor = self.hausart_kostenfaktoren.get(eingaben['hausart'], 1)  # Default: 1
        
        # Zusätzliche Faktoren (Architektenhaus, Makler, Denkmalschutz, Baujahr)
        architekt_aufschlag = 1.2 if eingaben['architektenhaus'] else 1  # 20% Aufschlag
        makler_aufschlag = 1.2 if eingaben['makler'] else 1  # 20% Aufschlag
        denkmalschutz_reduzierung = 0.75 if eingaben['denkmalschutz'] else 1  # 25% Reduzierung
        baujahr_reduzierung = 1 - (2024 - eingaben['baujahr']) * 0.001  # 0.1% Reduzierung pro Jahr
        
        # Berechnung des Gesamtkostenfaktors
        gesamtfaktor = (bundesland_faktor * lage_faktor * ausstattungs_faktor * 
                        hausart_faktor * architekt_aufschlag * makler_aufschlag * 
                        denkmalschutz_reduzierung * baujahr_reduzierung)
        
        # Gesamtpreis basierend auf den Faktoren
        gesamtpreis = (grundstück_preis + wohnfläche_preis) * gesamtfaktor
        
        return gesamtpreis

class PreisFinderApp:
    def __init__(self, root, schätzer):
        self.schätzer = schätzer
        root.title("Immobilien Preisfinder")
        root.geometry("400x500")
        root.resizable(0,0)
        self.create_label_entry(root, "Grundstück (m²):", "grundstück")
        self.create_label_entry(root, "Wohnfläche (m²):", "wohnfläche")

        self.architektenhaus_var = tk.BooleanVar()
        self.makler_var = tk.BooleanVar()
        self.denkmalschutz_var = tk.BooleanVar()
        
        self.create_checkbox(root, "Architektenhaus", self.architektenhaus_var)
        self.create_checkbox(root, "Makler beauftragen", self.makler_var)
        self.create_checkbox(root, "Denkmalschutz", self.denkmalschutz_var)

        self.lage_var = tk.StringVar()
        lage_optionen = ['Stadt', 'Land']
        self.create_dropdown(root, "Lage:", lage_optionen, self.lage_var)

        self.ausstattung_var = tk.StringVar()
        ausstattungs_optionen = ['Rohbau', 'Sanierungsbedarf', 'Renovierungsbedarf', 'Einfach', 'Gehoben']
        self.create_dropdown(root, "Ausstattung:", ausstattungs_optionen, self.ausstattung_var)

        self.baujahr_var = tk.StringVar()
        baujahr_optionen = [str(i) for i in range(1900, 2025)]
        self.create_dropdown(root, "Baujahr:", baujahr_optionen, self.baujahr_var)

        self.hausart_var = tk.StringVar()
        hausart_optionen = ['Einfamilienhaus', 'Doppelhaushälfte', 'Mehrfamilienhaus']
        self.create_dropdown(root, "Hausart:", hausart_optionen, self.hausart_var)

        self.bundesland_var = tk.StringVar()
        bundesland_optionen = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
        self.create_dropdown(root, "Bundesland:", bundesland_optionen, self.bundesland_var)

        self.berechnen_button = tk.Button(root, text="Preis berechnen", command=self.berechne_preis)
        self.berechnen_button.pack(pady=10)

        self.preis_label = tk.Label(root, text="Der geschätzte Preis wird hier angezeigt.")
        self.preis_label.pack(pady=10)

    def create_label_entry(self, root, text, var_name):
        label = tk.Label(root, text=text)
        label.pack()
        entry = tk.Entry(root)
        entry.pack()
        setattr(self, var_name + '_entry', entry)

    def create_checkbox(self, root, text, variable):
        checkbox = tk.Checkbutton(root, text=text, variable=variable)
        checkbox.pack()

    def create_dropdown(self, root, text, optionen, variable):
        label = tk.Label(root, text=text)
        label.pack()
        dropdown = ttk.Combobox(root, textvariable=variable, values=optionen)
        dropdown.pack()

    def berechne_preis(self):
        eingaben = {
            'grundstück': float(self.grundstück_entry.get()),
            'wohnfläche': float(self.wohnfläche_entry.get()),
            'architektenhaus': self.architektenhaus_var.get(),
            'makler': self.makler_var.get(),
            'denkmalschutz': self.denkmalschutz_var.get(),
            'lage': self.lage_var.get(),
            'ausstattung': self.ausstattung_var.get(),
            'baujahr': int(self.baujahr_var.get()),
            'hausart': self.hausart_var.get(),
            'bundesland': self.bundesland_var.get(),
        }
        preis = self.schätzer.berechne_preis(eingaben)
        self.preis_label.config(text=f"Geschätzter Preis: {preis:.2f} Euro")

if __name__ == "__main__":
    root = tk.Tk()
    schätzer = PreisSchätzer()
    app = PreisFinderApp(root, schätzer)
    root.mainloop()

oka

'coded by Ok@Nikolov'

**Erklärung des Backends und Frontends für die Praktikanten Eric, Artjoms und Michael**

*Der Python-Code erstellt eine Tkinter-Anwendung zur Berechnung von Immobilienpreisen. Die Berechnung basiert auf verschiedenen Kostenfaktoren, die in der Klasse PreisSchätzer definiert sind*

- **Klassen und Methoden:**
- *I: Klasse PreisSchätzer*

1. bundesland_kostenfaktoren: Ein Dictionary, das die Kostenfaktoren für verschiedene Bundesländer speichert. 
2. lage_kostenfaktoren: Ein Dictionary, das die Kostenfaktoren für die Lage (Stadt oder Land) speichert. 
3. ausstattungs_kostenfaktoren: Ein Dictionary, das die Kostenfaktoren für verschiedene Ausstattungen wie Rohbau, Sanierungsbedarf usw. speichert. 
4. hausart_kostenfaktoren: Ein Dictionary, das die Kostenfaktoren für verschiedene Hausarten wie Einfamilienhaus, Doppelhaushälfte usw. speichert. 
5. berechne_preis(self, eingaben): Diese Methode berechnet den Immobilienpreis auf Basis der übergebenen Eingaben. 

- **Die Berechnung umfasst:**
1. Grundstückspreis: Preis pro Quadratmeter Grundstück.
2. Wohnflächepreis: Preis pro Quadratmeter Wohnfläche.

**Kostenfaktoren:** Die Kostenfaktoren für Bundesland, Lage, Ausstattung und Hausart werden aus den Dictionaries abgerufen und in der Berechnung verwendet.
**Zusätzliche Faktoren:** Aufschläge und Reduzierungen für Architekten, Makler, Denkmalschutz und Baujahr werden berücksichtigt.

- *II: Klasse PreisFinderApp*

**__init__(self, root, schätzer):** Der Konstruktor der Klasse erstellt die Benutzeroberfläche mit Eingabefeldern für Grundstücksgröße, Wohnfläche, Lage, Ausstattung, Baujahr, Hausart und Bundesland.

1. create_label_entry: Methode zum Erstellen von Label-Entry-Paaren für Texteingaben.
2. create_checkbox: Methode zum Erstellen von Kontrollkästchen für optionale Eingaben (Architektenhaus, Makler, Denkmalschutz).
3. create_dropdown: Methode zum Erstellen von Dropdown-Menüs für Lage, Ausstattung, Baujahr, Hausart und Bundesland.
4. berechne_preis(self): Diese Methode wird aufgerufen, wenn der Benutzer auf den "Preis berechnen"-Button klickt. Sie sammelt die Eingaben aus der Benutzeroberfläche, berechnet den Preis unter Verwendung der PreisSchätzer-Instanz und zeigt den geschätzten Preis an.

*Der Hauptteil des Codes initialisiert das Tkinter-Fenster und die App:*

1. Ein Tkinter-Hauptfenster wird erstellt.
2. Eine Instanz der PreisSchätzer-Klasse wird initialisiert.
3. Eine Instanz der PreisFinderApp-Klasse wird gestartet.

**Zusammenfassung**
-Der Code erstellt eine grafische Benutzeroberfläche für die Berechnung von Immobilienpreisen unter Berücksichtigung verschiedener Kostenfaktoren. Die Berechnung wird durch die PreisSchätzer-Klasse durchgeführt, während die PreisFinderApp-Klasse die Benutzeroberfläche verwaltet.


**Quellen:**
Das Backend wurde mithilfe folgender Quellen erstellt
1. Eric's Unterricht
2. lerneprogrammieren.de (Letzter Zugriff 17.09.2024)
3. adesso.de (Letzter Zugriff 17.09.2024)
4. t2informatik.de (Letzter Zugriff 17.09.2024)
5. openai.com/chatgtp (Letzter Zugriff 17.09.2024)
6. Backend (FULL) Kurs via Youtube: https://www.youtube.com/watch?v=YyZzX2nWTwA (Letzter Zugriff 14.09.2024)

**Quellen:** Das Frontend wurde mithilfe folgender Quellen erstellt 
1. Eric's Unterricht
2. https://www.python-lernen.de/tkinter-farben-fg-bg.htm (letzter Zugriff 18.09.2024)
3. Python Kurs Tkinter via youtube: https://www.youtube.com/watch?v=oWrJD74KixA (letzter Zugriff 16.09.2024)
4. Objektorientierte Programmierungen https://www.youtube.com/watch?v=i1uQgU7NOCw (letzter Zugriff 16.09.2024)
5. openai.com/chatgtp  (Letzter Zugriff 17.09.2024)